In [1]:
from gensim.models import FastText

from parse_excel import ExcelParser
from ontology import Ontology

class Corpus:
    def __init__(self, dir, filename):
        self.dir = dir
        self.filename = filename

    def __iter__(self, ):
        parser = ExcelParser(self.dir)
        train_df = parser.read_train_set(self.filename)
        names = train_df['Полное наименование'].tolist()
        yield list(names)

def train_new(corpus, corpus_len):
    model = FastText(vector_size=4, window=3, min_count=1)
    model.build_vocab(corpus_iterable=corpus)
    model.train(corpus_iterable=corpus, total_examples=corpus_len, epochs=10)
    return model

def train_update(names):
    pass

onto = Ontology('ontology.owl')
parser = ExcelParser('docs')
train_df = parser.read_train_set('Обучающая_выборка.xlsx')

names = train_df['Полное наименование'].tolist()
names = [s.strip() for s in names]

classes = train_df['Класс (Наименование)'].tolist()
classes = [s.strip() for s in classes]

names_classes = dict(zip(names, classes)) 

corpus = Corpus('docs', 'Обучающая_выборка.xlsx')
model = train_new(corpus, len(names))
model.save('fasttext.model')

onto.create_many_classes(list(set(classes)))
onto.create_many_instances(names_classes)


* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [5]:

# Define class of new instance
wv = model.wv
topn = 3
most_sim = wv.most_similar("Диван 2-местный", topn=topn)

sim_classes = list()
for sim in most_sim:
    sim_classes.append(onto.get_class_of_instance(sim[0]))

set_sim_classes = set(sim_classes)

classes_count = list()
for cl in set_sim_classes:
    classes_count.append((cl, sim_classes.count(cl)))

classes_count = sorted(classes_count, key=lambda x: x[1], reverse=True)
print(classes_count)
perc_count = []
for cl in classes_count:
    perc_count.append((cl[0], (cl[1]/topn)*100))

print(perc_count)

[(ontology.Диван, 2), (ontology.Вода, 1)]
[(ontology.Диван, 66.66666666666666), (ontology.Вода, 33.33333333333333)]


In [3]:

train_df = parser.read_train_set('Обучающая_выборка2.xlsx')

names = train_df['Полное наименование'].tolist()
names = [s.strip() for s in names]

classes = train_df['Класс (Наименование)'].tolist()
classes = [s.strip() for s in classes]

names_classes = dict(zip(names, classes)) 

corpus = Corpus('docs', 'Обучающая_выборка2.xlsx')

model.build_vocab(corpus_iterable=corpus, update=True)
model.train(corpus_iterable=corpus, total_examples=len(names), epochs=model.epochs)

model.save('fasttext.model')

onto.create_many_classes(list(set(classes)))
onto.create_many_instances(names_classes)

In [1]:
from ontology import Ontology
    
onto = Ontology('ontology.owl')
classes = onto.get_all_class_names()
cl = onto.get_class_by_name('Вода')
lst = onto.get_all_instances_of(cl)
print(lst[0].is_a)

onto.create_instance(cl, 'Вода VOSS 0,3 стекло негаз')
inst = onto.get_all_instances_of(cl)
print(inst)
if 'TestClass' in classes:
    print('yes')

[ontology.Вода]
[ontology.Вода_Vittel_0,33_негаз, ontology.Вода VOSS 0,3 стекло негаз]
* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation
* Owlready2 * Warning: ignoring cyclic type of, involving storid 330

* Owlready2 * Warning: ignoring cyclic type of, involving storid 350



In [2]:
from ontology import Ontology

onto = Ontology('new_onto.owl')
sim = 'Вода_Vittel_0,33_негаз'
# type(onto.Вода_Vittel_0,33_негаз)
cl = onto.get_all_instances_of('Брошюровщик')

print(cl[0].name)
cl = onto.get_class_of_instance('Вода_Vittel_0,33_негаз')
print(cl)


Брошюровщик_Fellowes_Galaxy,_до_300мм,_до25л./до500л.,пл.пруж
new_onto.Вода
